## Vectorize words

In [1]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
import spacy
from gensim.models import Word2Vec
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')

/Users/caixinyi/CustomEmbed/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[nltk_data] Downloading package punkt to /Users/caixinyi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/caixinyi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [94]:
json_file_path = 'data/raw/450_all.json'

In [95]:
# combine the text together
with open(json_file_path, 'r') as file:
    json_data = json.load(file)
    subject_data = [entry['detail']['subject'] for entry in json_data]
    content_data = [entry['detail']['content'] for entry in json_data]
    post_data = [f"{subject} {content}" for subject, content in zip(subject_data, content_data)]
    answer_data = [answer['content'] for entry in json_data if entry['type'] == 'question' for answer in entry['answers']]
data = post_data + answer_data

# lowercase and tokenize the sentence
lower_sentence = [sentence.lower() for sentence in data]
tokenized_data = [
    [token for token in nltk.word_tokenize(sentence) if token.isalpha()]
    for sentence in lower_sentence
]
print(tokenized_data)

[['welcome', 'to', 'piazza', 'dear', 'students', 'we', 'welcome', 'you', 'to', 'the', 'piazza', 'board', 'here', 'you', 'are', 'free', 'to', 'post', 'any', 'questions', 'regarding', 'the', 'class', 'so', 'that', 'the', 'tas', 'and', 'zahid', 'can', 'answer', 'your', 'queries', 'remember', 'that', 'this', 'channel', 'is', 'additional', 'to', 'the', 'office', 'hours', 'and', 'emails', 'that', 'you', 'can', 'send', 'to', 'us', 'in', 'order', 'to', 'make', 'things', 'effective', 'and', 'streamlined', 'please', 'follow', 'these', 'on', 'the', 'top', 'of', 'the', 'dashboard', 'you', 'will', 'find', 'several', 'folders', 'ex', 'project', 'midterm', 'logistics', 'to', 'help', 'your', 'classmates', 'and', 'us', 'to', 'keep', 'track', 'of', 'them', 'make', 'sure', 'to', 'post', 'your', 'question', 'on', 'the', 'respective', 'make', 'sure', 'to', 'write', 'your', 'questions', 'as', 'clearly', 'as', 'possible', 'if', 'you', 'need', 'to', 'add', 'some', 'screenshots', 'or', 'other', 'information', 

In [96]:
# Term-frequency vectorizer method with spacy stop-word list

en = spacy.load('en_core_web_sm')
spacy_stop_words = list(en.Defaults.stop_words)
vectorizer = TfidfVectorizer(stop_words=spacy_stop_words)
tfidf_matrix = vectorizer.fit_transform(lower_sentence)

print(lower_sentence)

# Get feature names (words) with high TF-IDF scores
feature_names = vectorizer.get_feature_names_out()

# Words to be removed based on TF-IDF threshold
tfidf_threshold = 0.5  # Adjust as needed
stop_words = set([feature_names[i] for i, score in enumerate(tfidf_matrix.sum(axis=0).tolist()[0]) if score < tfidf_threshold])
stop_words.update(spacy_stop_words)

["welcome to piazza! dear ee450 students,we welcome you to the ee450 piazza board. here you are free to post any questions regarding the class so that the tas and prof. zahid can answer your queries. remember that this channel is additional to the office hours and emails that you can send to us. in order to make things effective and streamlined, please follow these guidelines:1. on the top of the dashboard, you will find several folders (ex: hw1, lab1, project, midterm, logistics, etc.). to help your classmates and us to keep track of them, make sure to post your question on the respective folder.2. make sure to write your questions as clearly as possible. if you need to add some screenshots or other information, please do so.3. before posting a question, make sure that you have checked previous posts so that we do not have a repeated question that has been already solved.4. we encourage you to respond unanswered questions from your classmates. learning is a collaborative work and any 

/Users/caixinyi/CustomEmbed/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(


In [5]:
custom_stop_words = {"need", "problem", "time", "like"}
stop_words.update(custom_stop_words)
print(stop_words)
print(len(stop_words))
filtered_data = [[word for word in sentence if word not in stop_words] for sentence in tokenized_data]


{'denote', 'mistakes', 'tool', 'capital', '184', 'introduced', 'folders4', 'coursecode', 'iperf3', 'with', 'clarifying', 'datagram', 'operate', 'nth', 'attempted', 'mostly', 'fact', 'difs', 'test_results', 'webex', 'realize', 'calls', '255', 'worth', '56355', 'emb', 'mix', 'possibly', 'essence', 'occurring', 'somebody', 'instruction', 'paper', 'recent', 'porfessor', 'x3', 'correspond', 'viazoomgagan', 'cwnd2', 'merry', 'increased', 'p24', 'programmatically', 'themselves', 'unale', 'dst', 'receving', '1k', 'had', 'bucket', 'instructors', 'bzero', 'seq', 'pacific', 'acked', '127', 'tricks', 'quit', 'htmlfollow', 'under', '2fkstgcobulb14t3', 'ai_flags', 'userguide', 'practices', 'grow', 'plan', 'area', 'stored', 'thetcp', 'unanswered', 'boxes', 'for', 'fixing', 'a8', 'discription', 'describe', 'decides', 'tcom370', 'pthreadsstudent', 'ensures', 'cvf', 'pick', 'cfvjwdrrnhc3cw9zz1pmevzxr0jfut09meeting', 'speed', '1t', 'hubs', 'europe', 'exponentially', 'clientandserver', 'fail_no_user', 'sn

In [101]:
# build the word2vec model
print(filtered_data)
print(len(filtered_data))
print([len(f) for f in filtered_data])
model = Word2Vec(filtered_data, vector_size=512, min_count=3)
model.save('word2vec_model_450.bin')

[['piazza', 'dear', 'students', 'piazza', 'board', 'free', 'post', 'questions', 'class', 'tas', 'zahid', 'answer', 'queries', 'remember', 'channel', 'additional', 'office', 'hours', 'send', 'order', 'things', 'follow', 'find', 'ex', 'project', 'midterm', 'help', 'sure', 'post', 'question', 'sure', 'write', 'questions', 'possible', 'add', 'screenshots', 'information', 'question', 'sure', 'previous', 'posts', 'repeated', 'question', 'respond', 'questions', 'learning', 'work', 'additional', 'help', 'share', 'homework', 'lab', 'project', 'piazza', 'taken', 'class', 'feel', 'free', 'add', 'question', 'reply', 'chinmay'], ['class', 'join', 'class', 'session'], ['chapter', 'says', 'pon', 'architecture', 'packets', 'sent', 'means', 'pon', 'shared', 'network', 'system', 'access', 'aon', 'switched', 'switch', 'different'], ['study', 'group', 'hi', 'interested', 'starting', 'study', 'know', 'helpful', 'people', 'learn', 'feel', 'interested', 'feel', 'free', 'send', 'email', 'start', 'group'], ['c

## Test vectorized model

In [99]:
def print_vector(model, word):
    word_vector = model.wv[word]
    print(f'Vector for {word}: ')
    print(word_vector)


def print_similar_words(model, word, num):
    similar_words = model.wv.most_similar(word, topn=num)
    print(f'Top {num} similar words to {word}: ')
    print(similar_words)


def print_similarity(model, word1, word2):
    cosine_similarity = model.wv.similarity(word1, word2)
    print(f'Similarity between {word1} and {word2}: ')
    print(cosine_similarity)


In [100]:
# Load the trained Word2Vec model
m = Word2Vec.load('word2vec_model_450.bin')

print_similarity(m, "tcp", "udp")
print_similarity(m, "student", "udp")
print_similar_words(m, 'udp', 10)
print_similar_words(m, 'network', 10)

Similarity between tcp and udp: 
0.9977243
Similarity between student and udp: 
0.98415184
Top 10 similar words to udp: 
[('client', 0.997833788394928), ('tcp', 0.9977242350578308), ('send', 0.9976084232330322), ('serverm', 0.9975574016571045), ('question', 0.9974934458732605), ('address', 0.9974839091300964), ('number', 0.9974711537361145), ('server', 0.9974404573440552), ('data', 0.9973480701446533), ('message', 0.9973389506340027)]
Top 10 similar words to network: 
[('server', 0.9979001879692078), ('tcp', 0.9978952407836914), ('client', 0.9978930354118347), ('use', 0.9978609681129456), ('question', 0.9977902770042419), ('address', 0.9977751970291138), ('case', 0.9975879192352295), ('course', 0.9975672364234924), ('send', 0.997564971446991), ('data', 0.997564971446991)]


## Clustering

In [16]:
# Load your trained Word2Vec model
model = Word2Vec.load("word2vec_model_450.bin")

# Get word vectors and corresponding words
word_vectors = model.wv.vectors
words = model.wv.index_to_key

print(len(words))

1018


In [8]:
# Number of clusters
# num_clusters = 10
num_clusters = len(words) // 10

In [9]:
# Apply k-means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=10)
clusters = kmeans.fit_predict(word_vectors)

word_cluster_mapping = dict(zip(words, clusters))
cluster_sums = {cluster_num: np.zeros_like(word_vectors[0]) for cluster_num in range(num_clusters)}
cluster_counts = {cluster_num: 0 for cluster_num in range(num_clusters)}

In [10]:
# Accumulate the sum of vectors and count for each cluster
for word, cluster in word_cluster_mapping.items():
    cluster_sums[cluster] += model.wv[word]
    cluster_counts[cluster] += 1

# Calculate the average vector for each cluster
cluster_averages = {cluster_num: cluster_sum / cluster_counts[cluster_num] for cluster_num, cluster_sum in cluster_sums.items()}

In [24]:
# Write  each cluster into cluster.txt
file_name = "clusters.txt"
with open(file_name, "w") as file:
    for cluster_num in range(num_clusters):
        cluster_words = [word for word, cluster in word_cluster_mapping.items() if cluster == cluster_num]
        # file.write(f"Cluster {cluster_num + 1}: {cluster_averages[cluster_num]} \n{cluster_words}\n")
        file.write(f"Cluster {cluster_num}: {cluster_words}\n")

## Customized doc2vec

In [37]:
# Tokenize the paragraph into words
def tokenize_paragraph(paragraph):
    return [
        [token for token in nltk.word_tokenize(sentence.lower()) if token.isalpha()]
        for sentence in nltk.sent_tokenize(paragraph)
    ]

In [21]:
# Function to find the closest cluster for a word
def find_closest_cluster(word):
    try:
        word_vector = model.wv[word]
        return kmeans.predict(word_vector.reshape(1, -1))[0]
    except KeyError:
        return -1

In [39]:
# Turn the words into their closest cluster
def doc2cluster(token):
    token_list = []
    for sentence in token:
        sentence_list = []
        for word in sentence:
            cluster_assignment = find_closest_cluster(word)
            if cluster_assignment != -1:
                sentence_list.append(cluster_assignment)
        token_list.append(sentence_list)
    return token_list

In [42]:
# Wrap the above process into a function
def build_histogram(token_sentence, num_c):
    if len(token_sentence) == 0:
        return
    histogram = [0]*num_c
    for word_cluster in token_sentence:
        histogram[word_cluster] += 1
    histogram_normalize = [count / len(token_sentence) for count in histogram]
    return histogram_normalize

In [40]:
# test the sample paragraph
sample_paragraph = "number of UDP sockets allowed to set up at serverM. As I was programming the serverM, I did not notice the identical config and wrote a socket for each other server. However, question @373 suggested one UDP socket is sufficient for all the three servers. May I keep the three UDP socket format?"

tokenized_sample_paragraph = tokenize_paragraph(sample_paragraph)
sample_paragraph_cluster = doc2cluster(tokenized_sample_paragraph)
print(sample_paragraph_cluster)
for sample_sentence_cluster in sample_paragraph_cluster:
    print(build_histogram(sample_sentence_cluster,num_clusters))

[[6, 70, 5, 90, 79, 18], [49, 18, 61, 63, 3, 15], [89, 70, 3, 44], [70, 3, 99]]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0

In [74]:
with open(json_file_path, 'r') as file:
    json_data = json.load(file)
    subject_data = [entry['detail']['subject'] for entry in json_data if entry['type'] == 'question']
    content_data = [entry['detail']['content'] for entry in json_data if entry['type'] == 'question']
    question_data = [f"{subject}. {content}" for subject, content in zip(subject_data, content_data)]
print(len(question_data))

file_name = "histogram.txt"
question_list = []
with open(file_name, "w") as file:
    for i, question in enumerate(question_data):
        question_clusters = doc2cluster(tokenize_paragraph(question))
        for j, question_cluster in enumerate(question_clusters):
            if len(question_cluster) != 0:
                question_list.append({"histo":build_histogram(question_cluster,num_clusters), "loc_q":i, "loc_s": j})

    for q in question_list:
        file.write(f"histogram: {q['histo']}\nlocation: {q['loc_q']}-{q['loc_s']}\n\n")

370


In [89]:
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_sentence(sentence_list, input_histo, num):
    input_histogram_array = np.array([input_histo])
    similarities = cosine_similarity([input_histo], [s['histo'] for s in sentence_list])
    similarities_list = similarities[0].tolist()
    sorted_histograms = sorted(zip(sentence_list, similarities_list), key=lambda x: x[1], reverse=True)
    closest_histograms = sorted_histograms[:num]
    return [f"{c[0]['loc_s']} in {c[0]['loc_q']} "for c in closest_histograms]
    # return closest_histograms

In [90]:
sample_question = "number of UDP sockets allowed to set up at serverM"
tokenized_sample_question = tokenize_paragraph(sample_question)
sample_question_cluster = doc2cluster(tokenized_sample_question)
sample_question_histo = build_histogram(sample_question_cluster[0],num_clusters)
find_similar_sentence(question_list, sample_question_histo,2)

['0 in 257 ', '1 in 228 ']

In [92]:
print(tokenize_paragraph(question_data[257])[0])

['number', 'of', 'udp', 'sockets', 'allowed', 'to', 'set', 'upat', 'serverm']
